In [ ]:
print("Starting Spark POOL Session")

from cloudbillingtool import all_billing

import pkg_resources
print( pkg_resources.get_distribution('cloudbillingtool').version )

In [ ]:
storage_account="cloudbillingtoolsa"
azure_data="abfss://azurebilling@"+storage_account+".dfs.core.windows.net/monthly/SenecBillingReport/*/*.csv"
hetzner_data="abfss://hetznerbilling@"+storage_account+".dfs.core.windows.net/*.csv"
metadata_dir="abfss://metadata@"+storage_account+".dfs.core.windows.net/"
output_dir="abfss://allbilling@"+storage_account+".dfs.core.windows.net"

In [ ]:
# compute the all_billing_data
all_billing_data = all_billing.generate_uniform_data_from(spark, azure_data, hetzner_data, "aws not implemented", metadata_dir )

In [ ]:
# For Debugging
#all_billing.toDF().show()
#all_billing_df = all_billing.toDF()
#all_billing_df.toPandas().head(100)

In [ ]:
all_billing_df = all_billing_data.toDF()
all_billing_df.write \
    .format("com.microsoft.kusto.spark.synapse.datasource") \
    .option("spark.synapse.linkedService", "AzureDataExplorerCloudBilling") \
    .option("kustoDatabase", "senecallbillingdb") \
    .option("kustoTable", "AllBilling") \
    .option("tableCreateOptions","CreateIfNotExist") \
    .mode("Append") \
    .save()

In [ ]:
from pyspark.sql.functions import explode
df_distinct_tags = all_billing_df.select( explode("CostResourceTag").alias("CostResourceTag") ).distinct()
    
df_distinct_tags.write \
    .format("com.microsoft.kusto.spark.synapse.datasource") \
    .option("spark.synapse.linkedService", "AzureDataExplorerCloudBilling") \
    .option("kustoDatabase", "senecallbillingdb") \
    .option("kustoTable", "CostResourceTags") \
    .option("tableCreateOptions","CreateIfNotExist") \
    .mode("Append") \
    .save()
